# Imports required for the data processing

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats.stats import pearsonr

# Read the processed CSV  file

The data_measurements.csv file generated in the previous stage is used as input at this stage

In [2]:
os.chdir("./data")
os.getcwd()

'C:\\Users\\sunee\\OneDrive\\Documents\\Georgia Tech\\MS Analytics\\CSE 6242\\Project\\latest\\git_gatech\\CODE\\data'

In [3]:
# Read  the csv file as pandas dataframe
df = pd.read_csv("data_measurements.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103291 entries, 0 to 103290
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   year              103291 non-null  int64  
 1   hdi               102256 non-null  float64
 2   inflation         102256 non-null  float64
 3   infant_mortality  102256 non-null  float64
 4   gdp               102256 non-null  float64
 5   gni               102256 non-null  float64
 6   forest            103246 non-null  float64
 7   population        102256 non-null  float64
 8   unemployment      102256 non-null  float64
 9   country_id        103291 non-null  int64  
 10  info              103291 non-null  object 
dtypes: float64(8), int64(2), object(1)
memory usage: 8.3+ MB


# Data Processing to generate correlation

Data flow process:
1. get a list of all the countries
2. for every country, calculate the pearson correlationship between forest percentage and other factors
3. create a consolidated dataframe for all countries and melt it into country, factor and factor_value

In [4]:
# get a list of all the countries 
countries = df['country_id'].drop_duplicates().tolist()

In [5]:
# reorder df coulmns to bring forst to the first index
corr_df = pd.DataFrame()
df = df[ ['forest'] + [ col for col in df.columns if col != 'forest' ] ]

In [6]:
#generate correlation between forest and other factors for all countries
for country in countries:
    df1 = df.loc[(df['country_id'] == country) & (df['year'] > 1990)]
    df_corr = df1.loc[:, ~df.columns.isin(['year', 'country_id'])].corr(method ='pearson')
    df_corr['country_id'] = country
    corr_df = corr_df.append(df_corr[:1])
corr_df.head()

,forest,hdi,inflation,infant_mortality,gdp,gni,population,unemployment,country_id
forest,1.0,2.038267e-16,9.023013e-17,-8.908434e-16,-1.605421e-16,2.562483e-17,2.352576e-16,1.416104e-14,139
forest,1.0,-3.957922e-01,9.269749e-02,4.014934e-01,-3.067132e-01,-4.154943e-01,5.529285e-01,-4.833261e-01,193
forest,1.0,-6.348305e-01,-4.010941e-01,7.800720e-02,-8.582995e-02,-2.974160e-02,3.542996e-01,-9.460660e-01,0
forest,1.0,3.446755e-01,NaN,2.529801e-01,2.721755e-02,1.241251e-01,6.826363e-01,4.798720e-01,96
forest,1.0,-1.356562e-01,2.729493e-01,1.604037e-02,-1.411389e-01,-2.934551e-01,-8.713459e-01,-9.220669e-01,148


In [7]:
corr_df = corr_df.fillna(0)
corr_df.reset_index(drop=True)

,forest,hdi,inflation,infant_mortality,gdp,gni,population,unemployment,country_id
0,1.0,2.038267e-16,9.023013e-17,-8.908434e-16,-1.605421e-16,2.562483e-17,2.352576e-16,1.416104e-14,139
1,1.0,-3.957922e-01,9.269749e-02,4.014934e-01,-3.067132e-01,-4.154943e-01,5.529285e-01,-4.833261e-01,193
2,1.0,-6.348305e-01,-4.010941e-01,7.800720e-02,-8.582995e-02,-2.974160e-02,3.542996e-01,-9.460660e-01,0
3,1.0,3.446755e-01,0.000000e+00,2.529801e-01,2.721755e-02,1.241251e-01,6.826363e-01,4.798720e-01,96
4,1.0,-1.356562e-01,2.729493e-01,1.604037e-02,-1.411389e-01,-2.934551e-01,-8.713459e-01,-9.220669e-01,148
...,...,...,...,...,...,...,...,...,...
202,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,124
203,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,161
204,1.0,0.000000e+00,1.000000e+00,-1.988250e-15,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,26
205,1.0,0.000000e+00,-1.000000e+00,4.316754e-16,0.000000e+00,0.000000e+00,0.000000e+00,5.814399e-16,83


In [8]:
corr_df[corr_df['country_id']==144]

,forest,hdi,inflation,infant_mortality,gdp,gni,population,unemployment,country_id
forest,1.0,0.0,-0.196582,-0.080037,0.018194,0.002107,0.009089,-0.30026,144


In [9]:
# melt the df to convert columns into rows
melted_df = corr_df.melt(id_vars=['country_id'], var_name='factor', value_name='measure')
melted_df = melted_df[melted_df['factor'] != 'forest']

In [10]:
melted_df.measure = melted_df.measure.round(2)

In [11]:
# generate factor rank based on correlation
melted_df['measure_abs'] = abs(melted_df['measure'])
melted_df['factor_rank'] = melted_df.groupby('country_id')['measure_abs'].rank(ascending=False)
melted_df.drop(['measure_abs'], axis =1, inplace = True)
melted_df['factor_rank'] = melted_df['factor_rank'].astype(int)

In [12]:
melted_df[melted_df['country_id'] == 144]

,country_id,factor,measure,factor_rank
376,144,hdi,0.00,6
583,144,inflation,-0.20,2
790,144,infant_mortality,-0.08,3
997,144,gdp,0.02,4
1204,144,gni,0.00,6
1411,144,population,0.01,5
1618,144,unemployment,-0.30,1


In [13]:
# function to generate  level of correlationship
def func(row):
    if abs(row['measure']) >= 0.7:
        return 'strong-linear'
    elif abs(row['measure']) >= 0.3:
        return 'mid-linear' 
    else:
        return 'weak-linear'


In [14]:
melted_df['level'] = melted_df.apply(func, axis=1)

In [15]:
melted_df['level'].drop_duplicates()

207      weak-linear
208       mid-linear
230    strong-linear
Name: level, dtype: object

In [16]:
melted_df

,country_id,factor,measure,factor_rank,level
207,139,hdi,0.00,4,weak-linear
208,193,hdi,-0.40,4,mid-linear
209,0,hdi,-0.63,2,mid-linear
210,96,hdi,0.34,3,mid-linear
211,148,hdi,-0.14,5,weak-linear
...,...,...,...,...,...
1651,124,unemployment,0.00,4,weak-linear
1652,161,unemployment,0.00,4,weak-linear
1653,26,unemployment,1.00,1,strong-linear
1654,83,unemployment,0.00,4,weak-linear


In [17]:
melted_df = melted_df[["country_id", "level", "factor", "measure", "factor_rank"]]
melted_df.reset_index(drop= True, inplace= True)
melted_df.head()

,country_id,level,factor,measure,factor_rank
0,139,weak-linear,hdi,0.00,4
1,193,mid-linear,hdi,-0.40,4
2,0,mid-linear,hdi,-0.63,2
3,96,mid-linear,hdi,0.34,3
4,148,weak-linear,hdi,-0.14,5


In [18]:
melted_df.to_csv("data_relationship.csv", index=False)

In [19]:
melted_df[melted_df['country_id'] == 116]

,country_id,level,factor,measure,factor_rank
129,116,weak-linear,hdi,-0.26,6
336,116,mid-linear,inflation,-0.51,3
543,116,mid-linear,infant_mortality,0.67,2
750,116,mid-linear,gdp,-0.38,4
957,116,weak-linear,gni,-0.28,5
1164,116,strong-linear,population,-0.89,1
1371,116,weak-linear,unemployment,0.19,7
